In [1]:
import casadi as ca
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches

from matplotlib import animation, rc
from IPython.display import HTML

# Inverted Pendulum on Cart

* [Dynamics Derivation](https://en.wikipedia.org/wiki/Inverted_pendulum)

In [2]:
def inv_pend_cart_ode():
    """
    Defines and ordinary differential equation for an inverted pendulum on a car.
    """
    
    M = ca.SX.sym('M')  # cart mass
    F = ca.SX.sym('F')  # force
    m = ca.SX.sym('m')  # pendulum mass
    g = ca.SX.sym('g')  # accel of gravity
    l = ca.SX.sym('l')  # length of pendulum
    
    # cart position and derivatives
    x = ca.SX.sym('x')  
    xd = ca.SX.sym('xd')
    xdd = ca.SX.sym('xdd')

    # pendulum angle and derivatives
    theta = ca.SX.sym('theta')
    thetad = ca.SX.sym('thetad')
    thetadd = ca.SX.sym('thetadd')

    # equations of motion as given in wiki link above
    eq1 = (M + m)*xdd - m*l*thetadd*ca.cos(theta) + m*l*thetad**2*ca.sin(theta) - F
    eq2 = l*thetadd - g*ca.sin(theta) -xdd*ca.cos(theta)
    eq_list = ca.vertcat(eq1, eq2)
    
    # this is a nonlinear system of equations, but we can solve for xdd, and thetadd using the jacobian, which is exact
    # since xdd and thetadd only appear linearly
    A = ca.jacobian(eq_list, ca.vertcat(thetadd, xdd))
    b = ca.substitute(eq_list, ca.vertcat(thetadd, xdd), ca.vertcat(0, 0))
    sol = ca.solve(A, b)
    ode = {}
    ode['x'] = ca.vertcat(theta, thetad, x, xd)  # state
    ode['p'] = ca.vertcat(m, M, l, g, F)  # parameters
    ode['ode'] = ca.vertcat(thetad, sol[0], xd, sol[1])  # right hand side X-dot
    return ode

In [3]:
def simulate_ode(ode, x0, p0, t0=0, tf=10, dt=0.01):
    """
    Simulates an ordinary differential equation.
    
    Paramers:
    ode: A dictionary describing the ode in the Casadi format.
    x0: Initial state.
    p0: The parameter vector, not inputs are also includeded here.
    t0: Initial time.
    tf: Final time.
    dt: The time-step for discrete updates and recording data.
    
    Return:
    data: A dictionary with keys for time: 't', and state: 'x'
    """
    t_vect = np.arange(t0, tf, dt)
    data = {
        't': [],
        'x': []
    }
    x = np.array(x0).reshape(-1)
    for t in t_vect:
        sim = ca.integrator('integrator', 'cvodes', ode, {'t0': t, 'tf': t + dt})
        res = sim(x0=x, p=p0)
        x = np.array(res['xf']).reshape(-1)
        data['t'].append(t)
        data['x'].append(x)

    for k in data.keys():
        data[k] = np.array(data[k])
    return data

In [4]:
def inv_pend_cart_anim(data):
    """
    Creates a simple animation of an inverted pendulum on a car.
    """
    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure()
    ax = plt.gca()

    ax.set_xlim(( -1, 4))
    ax.set_ylim((-1.5, 1.5))
    plt.axis('equal')

    line, = ax.plot([], [], lw=2)
    circle = patches.Circle((0, 0), 0.1, fc='none', ec='k')
    rect = patches.Rectangle((-0.5, 0), 0.2, 0.2, fc='none', ec='k')
    ax.add_artist(circle)
    ax.add_artist(rect)

    # initialization function: plot the background of each frame
    def init():
        return (line, circle, rect,)

    # animation function. This is called sequentially
    def animate(i):
        j = int(len(data['t'])/100*i)
        theta = data['x'][j, 0]
        x = data['x'][j, 2]
        rect.set_xy((x - 0.1, -0.1))
        line.set_data([[x, x - np.sin(theta)], [0, -np.cos(theta)]])
        circle.set_center((x - np.sin(theta), -np.cos(theta)))
        return (line,)

    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=100, interval=100, blit=True)
    plt.close()
    return anim

In [5]:
ode = inv_pend_cart_ode()
data = simulate_ode(ode, (3.14, 0, 0, 0), (1, 2, 1, 9.8, 0))
anim = inv_pend_cart_anim(data)
HTML(anim.to_html5_video())

In [6]:
ode

{'x': SX([theta, thetad, x, xd]),
 'p': SX([m, M, l, g, F]),
 'ode': SX(@1=(M+m), @2=(cos(theta)*(m*l)), @3=cos(theta), @4=((@2*@3)-(@1*l)), @5=(g*sin(theta)), @6=((((m*l)*sq(thetad))*sin(theta))-F), [thetad, (((@1/@4)*@5)-((@3/@4)*@6)), xd, (((@2/@4)*@5)-((l/@4)*@6))])}